In [1]:
import datasets
from datasets import load_dataset_builder, VerificationMode

/home/yuntaeyang_0629/anaconda3/envs/test_39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
builder = load_dataset_builder(
    './wmt_utils.py',
    language_pair = ('fr', 'en'),
    subsets = {
        datasets.Split.TRAIN: ["europarl_v7", "newscommentary_v10"],
        datasets.Split.VALIDATION: ['newstest2013'],
        datasets.Split.TEST: ['newstest2014']
    },

)

builder.download_and_prepare(verification_mode=VerificationMode.NO_CHECKS)

dataset = builder.as_dataset()

/home/yuntaeyang_0629/anaconda3/envs/test_39/lib/python3.9/site-packages/datasets/load.py:922: FutureWarning: The repository for wmt_utils contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at ./wmt_utils.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 2201939
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

In [24]:
def return_prompt_and_responses(samples):
    samples['text']="###input: " + samples['translation']['en'] + "###instruction: Please translate the input English sentence into French" + "###output:" + samples['translation']['fr']
    return samples

def return_prompt_and_responses_dpo(samples):
  samples['prompt'] = samples["translation"]['en']
  samples['chosen'] = samples["translation"]['fr']
  samples["rejected"] = samples["translation"]['fr']
  return samples

def load_and_prepare_dataset(source_dataset, train_types, data_types):
    if data_types=="train":
        dataset=source_dataset['train']
        original_columns = dataset.column_names
    elif data_types=="validation":
        dataset=source_dataset['validation']
        original_columns = dataset.column_names
    else:
        dataset=source_dataset['test']
        original_columns = dataset.column_names
        
    if train_types == "lora_sftt":
      dataset = dataset.map(
        return_prompt_and_responses
        )
      dataset = dataset.map( 
      batched=True,
      remove_columns=original_columns
      )
    elif train_types == "dpo":
      dataset = dataset.map(
        return_prompt_and_responses_dpo
        )
      dataset = dataset.map( 
      batched=True,
      remove_columns=original_columns
      )
    return dataset

In [25]:
new_dataset = load_and_prepare_dataset(dataset,"dpo", "validation")

Map: 100%|██████████| 3000/3000 [00:00<00:00, 36334.14 examples/s]
